**TL;DR**: do you just want to obtain the model and the training script? Run the two cells with `%%writefile`

# TensorFlow 2 and Keras Discovery

This is the very first demo run with TensorFlow. The goal of this notebook is to discover the very basics of TensorFlow, things like the different cells available in Keras and creating some easy NN.

## Creating the first Neural Network in TensorFlow 2 and Keras

In [ ]:
%%writefile model_def.py
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(13, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(6, activation=tf.nn.softmax)
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x1 = self.dense1(inputs)
        x2 = self.dense2(x1)
        y = self.out(x2)
        return y

Test: does it generate a prediction?

In [ ]:
from model_def import MyModel
from tensorflow.keras import Input

# Launch the DumbNet
model = MyModel()
# Create an input
i = Input(shape=(13,))
# Generate the output
out = model(i)
print(out)

In [ ]:
from tensorflow.keras.losses import MSE

# Loss definition. In this case, MSE (example)
target = Input(shape=(1,)) # random target value
print('Target: '+str(target))
loss = MSE(target, out) # Use MSE as Loss

print('Loss: '+str(loss))

Keras abstracts away the complexity of the training loop with the `fit` call. We will have:

```python
# Define the model and the optimizer
model = MyModel()
optimizer = tf.keras.optimizers.SGD(learning_rate)
# Compile the model with the optimizer and the chosen loss
model.compile(optimizer=optimizer, loss='mse')
# Train the model
model.fit(x_train, y_train, 
          batch_size=batch_size, epochs=epochs,
          validation_data=(x_test, y_test))
# evaluate on test set
scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
```

### Training script

The training script should:

1. Load the data via `np.load()` from our input
2. Set the hyperparameters from our inputs (epochs, batch_size, etc)
3. Choose the correct device
4. Load the model, the optimizer and compile it for a specific loss
5. Start the training with `fit()`
6. Evaluate results with `evaluate()`
7. Save the model with `save()`

In [ ]:
%%writefile train.py

import argparse
import numpy as np
import os
import tensorflow as tf

from model_def import MyModel

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


def parse_args():
    parser = argparse.ArgumentParser()
    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument('--epochs', type=int, default=1)
    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--learning_rate', type=float, default=0.1)
    # data directories
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    # model directory: we will use the default set by SageMaker, /opt/ml/model
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    return parser.parse_known_args()


def get_train_data(train_dir):
    x_train = np.load(os.path.join(train_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(train_dir, 'y_train.npy'))
    print('x train', x_train.shape,'y train', y_train.shape)
    return x_train, y_train

def get_test_data(test_dir):
    x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
    y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
    print('x test', x_test.shape,'y test', y_test.shape)
    return x_test, y_test

if __name__ == "__main__":
    # 0. Load the arguments
    args, _ = parse_args()
    # 1. Load the dataset
    print('Training data location: {}'.format(args.train))
    print('Test data location: {}'.format(args.test))
    x_train, y_train = get_train_data(args.train)
    x_test, y_test = get_test_data(args.test)
    # 2. Load the hyperparameters
    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate
    print('batch_size = {}, epochs = {}, learning rate = {}'.format(batch_size, epochs, learning_rate))
    # 3. Choose the device 
    device = '/cpu:0' 
    print(device)
    with tf.device(device):
        # 4. Load model, optimizer and compile
        model = MyModel()
        optimizer = tf.keras.optimizers.SGD(learning_rate)
        model.compile(optimizer=optimizer, loss='mse')    
        # 5. Train the model on the training dataset
        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                  validation_data=(x_test, y_test))
        # 6. Evaluate on test set
        scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
        print("\nTest MSE :", scores)
        # 7. Save the model
        model.save(args.model_dir + '/1')

Let's test it! 

In [ ]:
# Let's try loading data from the SKLearn Boston Housind dataset
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
x, y = data['data'], data['target']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
# Store them for later use
import os, numpy as np
local_dir = './'

np.save(os.path.join(local_dir, 'x_train.npy'), x_train)
np.save(os.path.join(local_dir, 'x_test.npy'), x_test)
np.save(os.path.join(local_dir, 'y_train.npy'), y_train)
np.save(os.path.join(local_dir, 'y_test.npy'), y_test)

Let's test the new training script (locally)!

In [ ]:
!python train.py \
    --epochs 5 \
    --batch_size 128 \
    --learning_rate 0.01 \
    --train . \
    --test . \
    --model_dir .